In [1]:
import numpy as np
from pulp import *
MEPS=1.03-6

In [2]:
def DEA_CCR(x,y,DMUs):
    m,n=x.shape
    s,n=y.shape
    
    res=[]
    for o in range(n):
        prob=LpProblem('DMU_'+str(o),LpMaximize)
        v=[LpVariable('v'+str(i),lowBound=0,
                      cat='Continuous') for i in range(m)]
        u=[LpVariable('u'+str(i),lowBound=0,
                      cat='Continuous') for i in range(s)]
        
        prob += lpDot(u,y[:,o])
        
        prob += lpDot(v,x[:,o])==1,'Normalize'str(o)
        for j in range(n):
            prob += lpDot(u,y[:,j]) <= lpDot(v,x[:,j])
            
        prob.solve()
        vs=np.array([v[i].varValue for i in range(m)])
        us=np.array([v[i].varValue for i in range(m)])
        # 参照集合作成
        (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))  <= MEPS)
        res.append([DMUs[o],value(prob.objectiove),
                    set(eo),tuple(vs),tuple(us)])
    return res

SyntaxError: invalid syntax (<ipython-input-2-773b3bbd2ed1>, line 15)